#### Lecture 1. Acceleration

- High-level IR --- DL compiler --- Low-level IR

#### mini code : list operations

In [1]:
import numpy as np
import time
size = 1000000
lst = range(size)
t1 = time.time()
result_list = [(a*b) for a,b in zip(lst, lst)]
print(round(time.time() - t1, 4), "seconds")

0.6235 seconds


In [3]:
array = np.arange(size)
t1 = time.time()
result_array = array * array
print(round(time.time() - t1, 4), "seconds")
array.dtype

0.0065 seconds


dtype('int32')

1. Acceleration
- python list vs numpy array
    - python list : 다른 data type을 가질 수 있음, element를 object로 저장
    - numpy array : densely packed in memory, C 언어로 함수 구현, 병렬 처리 가능
- python(interpreted language) vs C(compiled language)
- information transmission : 병목 현상 해결에 중점
- CPU design
    - bandwidth : data가 전송될 수 있는 capacity 개념
    - latency : packet이 network를 통과하는 데 걸리는 시간
    - throughput : data가 실제로 전송되는 양

2. Chip
- hardware acceleration : 특정 function에 대하여 일반적인 CPU보다 잘 수행하도록(latency 감소, throughput 증가) hardware 개선
    - ex) GPU(Graphic Processing Unit) / TPU(Tensor Processing Unit)
    - GPU : high compute density, parallel operations, high throughput
- ARM on IoT device
- SoC (System on Chip)

3. Compression & acceleration
- compression : space complexity(software) & acceleration : time complexity(hardware)
- hardware 설계 >> software 구현 >> compile 번역 >> user 입력 >> 실행 >> 출력 후 대기

4. Deep learning compiler
- HIgh level lanuage -(compiler)-> Assembly language -(assembler)-> machine language
- compiler 종류 : XLA(google), TVM(amazon), GLOW(facebook), ONNC
- MLIR - compiler를 여러가지 hardware 상황에 맞게 적용하는 부담을 줄여줌
- locality of reference
- hardware-aware compression


#### Lecture 2. Pruning

#### mini code : Masking

In [13]:
import numpy as np
from numpy.ma import array, masked_array, masked_values, masked_outside

vec1 = np.round(np.linspace(0.1, 1, 5), 3)
print('vec1:', vec1)
print('array:', array(vec1, mask = [0, 0, 0, 1, 0]))
print('masked_array:', masked_array(np.array(vec1), mask = [0, 0, 0, 1, 0]))
print('masked_outside:', masked_outside(vec1, 0.2, 0.9), '\n')

vec2 = [0, 1, 2, 3, 4, -9999]
print('vec2:', vec2)
print('masked_values:', masked_values(vec2, -9999))
print('masked_values:', masked_values(vec2, 3))
print('masked_values & filled:', masked_values(vec2, -9999).filled(5555))

vec1: [0.1   0.325 0.55  0.775 1.   ]
array: [0.1 0.325 0.55 -- 1.0]
masked_array: [0.1 0.325 0.55 -- 1.0]
masked_outside: [-- 0.325 0.55 0.775 --] 

vec2: [0, 1, 2, 3, 4, -9999]
masked_values: [0 1 2 3 4 --]
masked_values: [0 1 2 -- 4 -9999]
masked_values & filled: [   0    1    2    3    4 5555]


1. Weighted sum
- decision theory : weighted sum model(WSM)
- cross-entropy 도 weighted sum

2. Pruning 개념
- neuron 간 시냅스 연결의 일부를 잘라내는 것
- 얻는 것 : speed, 일반화 성능 / 잃는 것 : 정보 손실, hardware 메모리 적재 시 효율
- DL 에서 node 의 일부를 없앰 (drop out과 유사)
- dropout은 한번 잘라낸 weight를 재활용하는 과정을 random으로 반복하여 ensemble 효과를 주는 반면 pruning 에서는 inference time 에 복원하지 않음
- 가장 덜 중요한 neuron을 제거하고 fine-tuning 하는 과정을 반복
- pruning 이 가중치를 제거하는 방식이므로 L1, L2 regularization에 영향을 줌

3. Pruning 예
- 언제 : training 전 / 중 /후
- 방법 : weight 크기순, gradient-based, information-based
- 무엇을 : structured(channel, kernel, layer 기준) / unstructured(random weight 제거)
- iterative pruning - weight resetting / continued training

4. Lottery ticket hypothesis
- 랜덤하게 초기화된 dense NN 이 같은 학습 횟수안에 동일한 정확도를 갖는 sub-network가 존재할 것이라는 가설, 애초에 원래 모델을 학습 시켜야 비교군이 생긴다는 한계
- iterative magnitude pruning with rewinding

##### toy code
- pytorch에서는 pruning 시 원래의 가중치를 보관하고 있으면서 mask 행렬곱으로 pruning 을 구현하므로 size가 더 커지게 된다. 원래의 가중치가 필요없어지면 remove 명령어를 수행하면 mask와 원래의 가중치가 사라지고 mask 가 적용된 가중치로 변환됨